In [1]:
# Importar as bibliotecas necessárias

import os
from dotenv import load_dotenv
import pandas as pd
import psycopg2
from psycopg2 import OperationalError
import sqlalchemy
from sqlalchemy import create_engine
import panel as pn

In [ ]:
# Carregar as variáveis de ambiente do arquivo .env

load_dotenv()

In [2]:
# Pegar os valores das variáveis de ambiente

db_name = os.getenv('DB_NAME')
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')

database_url = os.getenv('DATABASE_URL')

In [3]:
try:
    # Tentando estabelecer a conexão com o banco de dados
    conn = psycopg2.connect(
        dbname=db_name,
        user=db_user,
        password=db_password,
        host=db_host,
        port=db_port,
        options="-c client_encoding=LATIN1"
    )
    print("Conexão estabelecida com sucesso!")
    
except OperationalError as e:
    # Captura de erros de conexão com o banco de dados
    print(f"Erro ao conectar ao banco de dados: {e}")

Conexão estabelecida com sucesso!


In [ ]:
engine = create_engine(database_url)

In [ ]:
pn.extension()
pn.extension('tabulator')
pn.extension(notifications=True)

In [ ]:
flag = ""

game_name = pn.widgets.TextInput(
    name="Nome do Jogo",
    value="",
    placeholder="Digite o nome do jogo",
    disabled=False
)

buttonSelect = pn.widgets.Button(name="Consultar", button_type="default")
# buttonInsert = pn.widgets.Button(name="Inserir", button_type="default")
# buttonDelete = pn.widgets.Button(name="Excluir", button_type="default")

In [ ]:
# Funções para manipular o banco de dados
def select_game():
    try:
        query = f"SELECT * FROM bd_jogos.jogo WHERE ('{game_name.value_input}'='{flag}' or nome = '{game_name.value_input}')"
        df = pd.read_sql(query, con=engine)
        table = pn.widgets.Tabulator(df, layout='fit_data')
        return table
    except Exception as e:
        print(e)
        return pn.pane.Alert('Não foi possível consultar!')

# def insert_game(event):
#     game_name = game_name.value
#     try:
#         query = f"INSERT INTO bd_jogos.jogo (name) VALUES ('{game_name}')"
#         with engine.connect() as connection:
#             connection.execute(query)
#         pn.state.notifications.success(f"Jogo '{game_name}' inserido com sucesso!")
#     except Exception as e:
#         pn.state.notifications.error(f"Erro ao inserir: {e}")

# def delete_game(event):
#     game_name = game_name.value
#     try:
#         query = f"DELETE FROM bd_jogos.jogo WHERE name = '{game_name}'"
#         with engine.connect() as connection:
#             connection.execute(query)
#         pn.state.notifications.success(f"Jogo '{game_name}' excluído com sucesso!")
#     except Exception as e:
#         pn.state.notifications.error(f"Erro ao excluir: {e}")


In [ ]:
def table_creator(cons):
    if cons:
        return select_game()

interactive_table = pn.bind(table_creator, buttonSelect)

In [ ]:
layout = pn.Column(
    game_name,
    buttonSelect,
    interactive_table
).servable()
